#Import the initial library which will be use in extraction

In [1]:
from PIL import Image
from pytesseract import image_to_string
import pytesseract
import numpy as np
import pandas as pd
import PIL
import cv2
import re
import glob


#Location of Tesseract in your system

In [2]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

#Initial coordinates of s.no,voter id,section and part number and whole block of name, guardian's relation and name,house number,age,gender. Please edit and change by hit and trial basis.

In [3]:
def image_text(x,crop_rectangle_box = (126,275, 790, 540),crop_rectangle_sn= (215,235, 321, 277),
               crop_rectangle_vid= (330,230,1032, 280),crop_rectangle_sect = (508,100, 3000, 180),
               crop_rectangle_pn = (2670,20, 2900, 100), crop_rectangle_ac = (730,20, 1400, 80),
               crop_rectangle_pg = (2720,3750, 2800,3880),crop_rectangle_dt = (1650,3750, 1850,3880)):
    im = Image.open(x)
    im = im.resize((3000, 3882), PIL.Image.LANCZOS)
    im = cv2.bilateralFilter(np.array(im, dtype= np.uint8),9,75,75)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im = cv2.adaptiveThreshold(np.array(im,dtype=np.uint8), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 9)
    im = Image.fromarray(im)
    countt = 1 
    
    for i in range(0,10):
        for j in range(0,3):
            kk = []
            
            cropped_im_box = im.crop(crop_rectangle_box)
            c_box = image_to_string(cropped_im_box,config='-c preserve_interword_spaces=1',lang='hin')
            c_box = re.sub(r'\n\s*\n','\n',c_box,re.MULTILINE)
            details = c_box.splitlines()
            
            for i in details:
                if len(i)>0:
                    if len(i.split(":"))>1:
                        kk.append(i)

            if len(kk)==4:
                file_name.append(x+str(countt))
                n = kk[0].strip().split(":")[1]
                n = re.sub("^\s+|\s+$", "", n, flags=re.UNICODE)
                f = kk[1].strip().split(":")[1]
                f = re.sub("^\s+|\s+$", "", f, flags=re.UNICODE)
                h = kk[2].strip().split(":")[1]
                mm = kk[3].strip().split(":")
                a = mm[1].strip().split(" ")[0]
                g = mm[-1].strip()
                gr = kk[1].strip().split(":")[0]
                gr = gr.strip().split(' ')[0]
                name.append(n)
                father.append(f)
                hno.append(h)
                age.append(a)
                gen.append(g)
                grel.append(gr)
                
            else:
            
                return image_text_2(im,x)
            
            cropped_im_sn = im.crop(crop_rectangle_sn)
            c_sn = image_to_string(cropped_im_sn, config='-c tessedit_char_whitelist= 0123456789 --psm 6')
            c_sn = re.sub("^\s+|\s+$", "", c_sn, flags=re.UNICODE)
            sn.append(c_sn)
            
            cropped_im_pn = im.crop(crop_rectangle_pn)
            c_pn = image_to_string(cropped_im_pn, config='-c tessedit_char_whitelist=0123456789 --psm 6')
            c_pn = re.sub("^\s+|\s+$", "", c_pn, flags=re.UNICODE)
            part.append(c_pn)
            
            cropped_im_vid = im.crop(crop_rectangle_vid)
            c_vid = image_to_string(cropped_im_vid,config='-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 --psm 6')
            c_vid = re.sub("^\s+|\s+$", "", c_vid, flags=re.UNICODE)
            vid.append(c_vid)
            
            cropped_im_sect = im.crop(crop_rectangle_sect)
            c_sect = image_to_string(cropped_im_sect,config='-c preserve_interword_spaces=1',lang='hin')
            c_sect = re.sub("^\s+|\s+$", "", c_sect, flags=re.UNICODE)
            section.append(c_sect)
            
            cropped_im_ac = im.crop(crop_rectangle_ac)
            c_ac = image_to_string(cropped_im_ac,config='-c preserve_interword_spaces=1', lang='hin')
            c_ac = re.sub("^\s+|\s+$", "", c_ac, flags=re.UNICODE)
            ac.append(c_ac)
            
            cropped_im_pg = im.crop(crop_rectangle_pg)
            c_pg = image_to_string(cropped_im_pg, config='-c tessedit_char_whitelist=0123456789 --psm 6')
            c_pg = re.sub("^\s+|\s+$", "", c_pg, flags=re.UNICODE)
            pg.append(c_pg)
            
            cropped_im_dt = im.crop(crop_rectangle_dt)
            c_dt = image_to_string(cropped_im_dt, config='-c tessedit_char_whitelist=\-0123456789 --psm 6')
            c_dt = re.sub("^\s+|\s+$", "", c_dt, flags=re.UNICODE)
            dt.append(c_dt)
            
            countt+=1

# Find distance between same category horizontally.
# For example find distance between starting coordinate of first box NAME and starting coordinate of secnd box NAME on right
# Make sure we find distance from starting point. Distance should be same for 2nd and 3rd box horizontally as well as for every 4 other value, such as age, sex etc
            crop_rectangle_box = tuple(np.add(crop_rectangle_box, (912, 0,912,0)))
            crop_rectangle_sn = tuple(np.add(crop_rectangle_sn, (912, 0,912,0)))
            crop_rectangle_vid =  tuple(np.add(crop_rectangle_vid, (912, 0,912,0)))
# Find ditance between third and first box. Which should be 3 times above found distance

        crop_rectangle_box = tuple(np.subtract(crop_rectangle_box, (2736, 0,2736,0)))
        crop_rectangle_sn = tuple(np.subtract(crop_rectangle_sn, (2736, 0,2736,0)))
        crop_rectangle_vid =  tuple(np.subtract(crop_rectangle_vid, (2736, 0,2736,0)))

# Find distance between same category vertically.
# For example find distance between starting coordinate of first box NAME and starting coordinate of secnd box NAME down
# Distance should be same for all 10 boxes vertically as well as for every 4 other value, such as age, sex etc
        crop_rectangle_box = tuple(np.add(crop_rectangle_box, (0, 325,0,325)))
        crop_rectangle_sn = tuple(np.add(crop_rectangle_sn, (0, 324,0,324)))
        crop_rectangle_vid =  tuple(np.add(crop_rectangle_vid, (0, 324,0,324)))


    return name,father,hno,age,gen,file_name,grel,sn,vid,section,part, ac,pg,dt

In [4]:
def image_text_2(im,x):
    crop_rectangle_box = (126,234,790, 500)
    crop_rectangle_sn= (235, 195, 320, 230)
    crop_rectangle_vid= (535,195, 1017, 245)
    crop_rectangle_sect= (508,100, 3000, 180)
    crop_rectangle_pn = (2640,20, 2900, 100)
    crop_rectangle_ac = (730,20, 1400, 80)
    crop_rectangle_pg = (2725,3750, 2800,3880)
    crop_rectangle_dt = (1650,3750, 1850,3880)
    
    countt = 1 
    
    for i in range(0,10):
        for j in range(0,3):
            kk = []
            cropped_im_box = im.crop(crop_rectangle_box)
            c_box = image_to_string(cropped_im_box,config='-c preserve_interword_spaces=1',lang='hin')
            c_box = re.sub(r'\n\s*\n','\n',c_box,re.MULTILINE)
            details = c_box.splitlines()
           
            for i in details:
                if len(i)>0:
                    if len(i.split(":"))>1:
                        kk.append(i)
            
            if len(kk) ==4:
                file_name.append(x+str(countt))
                n = kk[0].strip().split(":")[1]
                n = re.sub("^\s+|\s+$", "", n, flags=re.UNICODE)
                f = kk[1].strip().split(":")[1]
                f = re.sub("^\s+|\s+$", "", f, flags=re.UNICODE)
                h = kk[2].strip().split(":")[1]
                mm = kk[3].strip().split(":")
                a = mm[1].strip().split(" ")[0]        
                g = mm[-1].strip()
                gr = kk[1].strip().split(":")[0]
                gr = gr.strip().split(' ')[0]
                name.append(n)
                father.append(f)
                hno.append(h)
                age.append(a)
                gen.append(g)
                grel.append(gr)
              
                    
                
            else:
                file_name.append("Delete")
                name.append("Delete")
                father.append("Delete")
                hno.append("Delete")
                age.append("Delete")
                gen.append("Delete")
                grel.append("Delete")
                
                
            cropped_im_sn = im.crop(crop_rectangle_sn)
            c_sn = image_to_string(cropped_im_sn, config='-c tessedit_char_whitelist=0123456789 --psm 6')
            c_sn = re.sub("^\s+|\s+$", "", c_sn, flags=re.UNICODE)
            sn.append(c_sn)
            
            cropped_im_pn = im.crop(crop_rectangle_pn)
            c_pn = image_to_string(cropped_im_pn, config='-c tessedit_char_whitelist=0123456789 --psm 6')
            c_pn = re.sub("^\s+|\s+$", "", c_pn, flags=re.UNICODE)
            part.append(c_pn)
            
            cropped_im_vid = im.crop(crop_rectangle_vid)
            c_vid = image_to_string(cropped_im_vid,config='-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 --psm 6')
            c_vid = re.sub("^\s+|\s+$", "", c_vid, flags=re.UNICODE)
            vid.append(c_vid)
            
            cropped_im_sect = im.crop(crop_rectangle_sect)
            c_sect = image_to_string(cropped_im_sect,config='-c preserve_interword_spaces=1', lang='hin')
            c_sect = re.sub("^\s+|\s+$", "", c_sect, flags=re.UNICODE)
            section.append(c_sect)
            
            cropped_im_ac = im.crop(crop_rectangle_ac)
            c_ac = image_to_string(cropped_im_ac,config='-c preserve_interword_spaces=1', lang='hin')
            c_ac = re.sub("^\s+|\s+$", "", c_ac, flags=re.UNICODE)
            ac.append(c_ac)
            
            cropped_im_pg = im.crop(crop_rectangle_pg)
            c_pg = image_to_string(cropped_im_pg, config='-c tessedit_char_whitelist=0123456789 --psm 6')
            c_pg = re.sub("^\s+|\s+$", "", c_pg, flags=re.UNICODE)
            pg.append(c_pg)
            
            cropped_im_dt = im.crop(crop_rectangle_dt)
            c_dt = image_to_string(cropped_im_dt, config='-c tessedit_char_whitelist=\-0123456789 --psm 6')
            c_dt = re.sub("^\s+|\s+$", "", c_dt, flags=re.UNICODE)
            dt.append(c_dt)
            
            countt+=1
                

# Find distance between same category horizontally.
# For example find distance between starting coordinate of first box NAME and starting coordinate of secnd box NAME on right
# Make sure we find distance from starting point. Distance should be same for 2nd and 3rd box horizontally as well as for every 4 other value, such as age, sex etc
            crop_rectangle_box = tuple(np.add(crop_rectangle_box, (912, 0,912,0)))
            crop_rectangle_sn = tuple(np.add(crop_rectangle_sn, (912, 0,912,0)))
            crop_rectangle_vid =  tuple(np.add(crop_rectangle_vid, (912, 0,912,0)))

# Find ditance between third and first box. Which should be 3 times above found distance

        crop_rectangle_box = tuple(np.subtract(crop_rectangle_box, (2736, 0,2736,0)))
        crop_rectangle_sn = tuple(np.subtract(crop_rectangle_sn, (2736, 0,2736,0)))
        crop_rectangle_vid =  tuple(np.subtract(crop_rectangle_vid, (2736, 0,2736,0)))


# Find distance between same category vertically.
# For example find distance between starting coordinate of first box NAME and starting coordinate of secnd box NAME down
# Distance should be same for all 10 boxes vertically as well as for every 4 other value, such as age, sex etc
        crop_rectangle_box = tuple(np.add(crop_rectangle_box, (0, 325,0,325)))
        crop_rectangle_sn = tuple(np.add(crop_rectangle_sn, (0, 324,0,324)))
        crop_rectangle_vid =  tuple(np.add(crop_rectangle_vid, (0, 324,0,324)))



    return name,father,hno,age,gen,file_name,grel,sn,vid,section,part,ac,pg,dt

#Fetch all data from the location

In [5]:
#Getting all page 1
page_1 = glob.glob('*-01.jpeg')
#Getting all page 2
page_2 = glob.glob('*-02.jpeg')
#Getting all pages
page_all = glob.glob('*.jpeg')

#Removing page 1 and 2 from all

In [6]:
temp = [i for i in page_all if i not in page_1]
page_fin = [i for i in temp if i not in page_2]

#It gives information about how many pages for extraction

In [7]:
len(page_fin)

34

In [8]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

#Splitting files into 400 parts to increase frequency of saving. Increase the value to save more frequently. Do not change value after running code once. If want to change value after running. Delete all saved files or do maths

In [9]:
page_fin_split =  list(split(page_fin,1))

In [10]:
count = 1
count_sem = 1
for i in page_fin_split[count-1:]:
    name = []
    father = []
    hno = []
    age = []
    gen = []
    file_name = []
    grel =[]
    sn= []
    vid =[]
    section= []
    part =[]
    ac = []
    pg = []
    dt = []
    for j in i:
        n,f,h,a,g,fn,gr,s,v,sc,p,q,pg,dt = image_text(j)
        print("page_"+str(count_sem))
        count_sem+=1
    
    df = pd.DataFrame(np.column_stack([q,p,sc,s,v,n,gr,f,h,a,g,pg,dt,fn]), 
                               columns=['AssemblyConstituency','PartNum','Section','SNo','Voter ID',
                                        'Name','RelationType','RelationName', 'HouseNum','Age','Gender',
                                        'PageNum','Date','FileName'])
    df.to_excel(str(count)+".xlsx",index = False)
    print(count)
    count+=1

page_1
page_2
page_3
page_4
page_5
page_6
page_7
page_8
page_9
page_10
page_11
page_12
page_13
page_14
page_15
page_16
page_17
page_18
page_19
page_20
page_21
page_22
page_23
page_24
page_25
page_26
page_27
page_28
page_29
page_30
page_31
page_32
page_33
page_34
1


In [11]:
df.head()

,AssemblyConstituency,PartNum,Section,SNo,Voter ID,Name,RelationType,RelationName,HouseNum,Age,Gender,PageNum,Date,FileName
0,68-सितारगंज,21,"रूदपुर, पो० ओ० - सितारगंज, तहसील - सितारगंज",241,GRP2818748,गोलक मण्डल,पिता,खोकन,28/2,43,पुरुष,11,07-02-2020,10.jpeg1
1,68-सितारगंज,21,"रूदपुर, पो० ओ० - सितारगंज, तहसील - सितारगंज",242,GRP2814945,सलोक मण्डल,पति,केना राम,28/3,43,महिला,11,07-02-2020,10.jpeg2
2,68-सितारगंज,21,"रूदपुर, पो० ओ० - सितारगंज, तहसील - सितारगंज",243,WPH0636993,प्रशान्त मण्डल,पिता,पंचानन,28/3,43,पुरुष,11,07-02-2020,10.jpeg3
3,68-सितारगंज,21,"रूदपुर, पो० ओ० - सितारगंज, तहसील - सितारगंज",244,WPH0637009,रीना मण्डल,पति,प्रशान्त मण्डल,28/3,4,महिला,11,07-02-2020,10.jpeg4
4,68-सितारगंज,21,"रूदपुर, पो० ओ० - सितारगंज, तहसील - सितारगंज",245,GRP2814754,चम्पा मण्डल,पति,गोलक,28/3,4॥,महिला,11,07-02-2020,10.jpeg5


In [12]:
df.shape

(1027, 14)